<a href="https://colab.research.google.com/github/GeorgeCrossIV/Product-Recommendation-Chatbot/blob/main/Vector_Search_Product_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started with this notebook

- Create a new vector search enabled database in Astra. [astra.datastax.com](https://astra.datastax.com)
- For the easy path, name the keyspace in that database "vector_preview" (otherwise be prepared to modify the CQL in this notebook)
- Create a token with permissions to create tables
- Download your secure-connect-bundle zip file.
- Download the [sample data file from here](https://drive.google.com/file/d/1KlXnYy6CECoQz7wjf-728ci_unpMSxvF/view?usp=sharing)
- When you open this notebook in Google Colab or your own notebook server, drag-and-drop the secure connect bundle and ProductDataset.csv into the File Browser of the notebook
- Set up an open.ai API account and generate a key
- Update the Keys & Environment Variables cell in the notebook with information from the token you generated and the name of your secure connect bundle file.

# Setup

In [ ]:
!pip install openai pandas jupyter-datatables cassandra-driver

# Imports

In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory
from cassandra.query import SimpleStatement
from google.colab import userdata
import openai
import numpy
import pandas as pd

# Keys & Environment Variables

In [3]:
# keys and tokens here
openai_api_key = userdata.get('openai_api_key')
openai.api_key = openai_api_key
cass_user = userdata.get('cass_user')
cass_pw = userdata.get('cass_pw')
scb_path = '/content/secure-connect-cassio-db.zip'
keyspace="chatbot"
table="products_table"

# Select a model to compute embeddings

In [4]:
model_id = "text-embedding-ada-002"

# Connect to the Cluster

In [5]:
cloud_config= {
  'secure_connect_bundle': scb_path
}
auth_provider = PlainTextAuthProvider(cass_user, cass_pw)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider, protocol_version=4)
session = cluster.connect()
session.set_keyspace('vector_preview')
session

# Drop / Create Schema

In [ ]:
# only use this to reset the schema
session.execute(f"""DROP INDEX IF EXISTS {keyspace}.openai_desc""")
session.execute(f"""DROP TABLE IF EXISTS {keyspace}.{table}""")

In [ ]:
# # Create Table
session.execute(f"""
CREATE TABLE IF NOT EXISTS {keyspace}.{table}
(product_id int,
 chunk_id int,

 product_name text,
 description text,
 price text,

 openai_description_embedding vector<float, 1536>,
 minilm_description_embedding vector<float, 384>,

 PRIMARY KEY (product_id, chunk_id))
 """)

# # Create Index
session.execute(f"""CREATE CUSTOM INDEX IF NOT EXISTS openai_desc ON {keyspace}.{table} (openai_description_embedding) USING 'org.apache.cassandra.index.sai.StorageAttachedIndex'""")

# Load the table with data and create text embeddings

In [ ]:
 !wget https://raw.githubusercontent.com/GeorgeCrossIV/Langchain-Retrieval-Augmentation-with-CASSIO/main/20220301.simple.csv

In [6]:
products_list = pd.read_csv('ProductDataset.csv')
products_list

,product_id,product_name,description,price
0,552,Sony Turntable - PSLX350H,Sony Turntable - PSLX350H/ Belt Drive System/ ...,NaN
1,580,Bose Acoustimass 5 Series III Speaker System -...,Bose Acoustimass 5 Series III Speaker System -...,$399.00
2,4696,Sony Switcher - SBV40S,Sony Switcher - SBV40S/ Eliminates Disconnecti...,$49.00
3,5644,Sony 5 Disc CD Player - CDPCE375,Sony 5 Disc CD Player- CDPCE375/ 5 Disc Change...,NaN
4,6284,Bose 27028 161 Bookshelf Pair Speakers In Whit...,Bose 161 Bookshelf Speakers In White - 161WH/ ...,$158.00
...,...,...,...,...
1076,39088,Logitech Cordless Desktop Wave Keyboard And Mo...,Logitech Cordless Desktop Wave Keyboard And Mo...,$79.00
1077,39090,Mitsubishi DLP Black TV Stand - MBS73V,Mitsubishi DLP Black TV Stand - MBS73V/ Matchi...,$549.00
1078,39175,Logitech Digital Precision PC Gaming Headset -...,Logitech Digital Precision PC Gaming Headset -...,$49.00
1079,39176,Logitech 2.1 Multimedia Silver Speaker System ...,Logitech 2.1 Multimedia Silver Speaker System ...,NaN


In [ ]:
for id, row in products_list.iterrows():
  # Create Embedding for each conversation row, save them to the database
  text_chunk_length = 2500
  text_chunks = [row.description[i:i + text_chunk_length] for i in range(0, len(row.description), text_chunk_length)]
  for chunk_id, chunk in enumerate(text_chunks):
    pricevalue = row.price if isinstance(row.price, str) else ""
    full_chunk = f"{chunk} price: {pricevalue}"
    embedding = openai.embeddings.create(input=full_chunk, model=model_id).data[0].embedding
    query = SimpleStatement(
                f"""
                INSERT INTO {keyspace}.{table}
                (product_id, chunk_id, product_name, description, price, openai_description_embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
            )
    display(row)

    session.execute(query, (row.product_id, chunk_id, row.product_name, row.description, pricevalue, embedding))



---


# Start using the index

In the steps up to this point, we have been creating a schema and loading the table with data, including embeddings we generated through the OpenAI Embedding API.
Now we are going to query that table and use the results to give ChatGPT some context to support it's response.

# Convert a query string into a text embedding to use as part of the query

This is where the real fun starts.  Provide a question or request to be used as the query.  The source sample database is mostly consumer electronics and appliances, so imagine you're talking to a customer service rep at Best Buy or another electronics store.

Here we use the same API that we used to calculate embeddings for each row in the database, but this time we are using your input question to calculate a vector to use in a query.

In [ ]:
customer_input = "What equipement would you recommend for a computer workstation setup costing less than $2000?"

embedding = openai.embeddings.create(input=customer_input, model=model_id).data[0].embedding
display(embedding)

Let's take a look at what a query against a vector index could look like.  The query vector has the same dimensions (number of entries in the list) as the embeddings we generated a few steps ago for each row in the database.

In [ ]:
query = SimpleStatement(
    f"""
    SELECT *
    FROM {keyspace}.{table}
    ORDER BY openai_description_embedding ANN OF {embedding} LIMIT 5;
    """
    )
display(query)

# Find the top 5 results using ANN Similarity

In [10]:
results = session.execute(query)
top_5_products = results._current_rows

for row in top_5_products:
  print(f"""{row.product_id}, {row.product_name}, {row.description}, {row.price}\n""")

37812, Sony VAIO RT Series Black All-In-One Desktop Computer - VGCRT150Y, Sony VAIO RT Series Black All-In-One Desktop Computer - VGCRT150Y/ 2.66GHz Intel Core 2 Quad Q9400 Processor/ 8GB Memory/ Built-In CompactFlash Media Slot/ 1TB Serial ATA Hard Drive/ 25.5' XBRITE-Full HD LCD/ Integrated Stereo A2DP Bluetooth Technology/ 6MB L2 Cache/ Microsoft Windows Vista Ultimate 64-Bit/ Black Finish, $3,999.00

38400, Sony VAIO JS Series Black All-In-One Desktop Computer - VGCJS130JB, Sony VAIO JS Series Black All-In-One Desktop Computer - VGCJS130JB/ 2.5GHz Intel Pentium Dual-Core Processor E5200/ 20.1' (1680 x 1050) Widescreen WSXGA+ XBRITE-HiColor Technology Display/ 500GB Serial ATA 7200rpm Hard Drive/ Built-In 1.3 Megapixel MOTION EYE Camera And Microphone With Face-Tracking Technology/ 4GB PC2-6400 (2GBx2) Installed Memory/ 800MHz Front Side Bus Speed/ 2MB L2 Cache/ Genuine Microsoft Windows Vista Home Premium 64-Bit/ Black Finish, $1,099.00

37877, Apple MacBook Pro 2.4GHz Intel Core 2

# Ask ChatGPT for some help (Prompt Engineering)

- Here we build a prompt with which we'll query ChatGPT.  Note the "roles" in this little conversation give the LLM more context about who that part of the conversation is coming from.
- This may take 10-20 seconds to return, so be patient.

In [11]:

message_objects = []
message_objects.append({"role":"system",
                        "content":"You're a chatbot helping customers with questions and helping them with product recommendations"})

message_objects.append({"role":"user",
                        "content": customer_input})

message_objects.append({"role":"user",
                        "content": "Please give me a detailed explanation of your recommendations"})

message_objects.append({"role":"user",
                        "content": "Please be friendly and talk to me like a person, don't just give me a list of recommendations"})

message_objects.append({"role": "assistant",
                        "content": "I found these 3 products I would recommend"})

products_list = []

for row in top_5_products:
    brand_dict = {'role': "assistant", "content": f"{row.description}"}
    products_list.append(brand_dict)

message_objects.extend(products_list)
message_objects.append({"role": "assistant", "content":"Here's my summarized recommendation of products, and why it would suit you:"})

completion = openai.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=message_objects
)
print(completion.choices[0].message.content)


For a computer workstation setup costing less than $2000, I would highly recommend the following:

1. Sony VAIO LV Series Silver All-In-One Desktop Computer - VGCLV150J:
   - It has a powerful 2.53GHz Intel Core 2 Duo E7200 Processor, ensuring smooth performance for your tasks.
   - Comes with 4GB of memory, providing sufficient multitasking capabilities.
   - Offers a 500GB Serial ATA hard drive, giving you plenty of storage space for your files.
   - The 24" XBRITE-Full HD LCD screen provides a vibrant and immersive visual experience.
   - It has an integrated stereo A2DP Bluetooth technology for easy connectivity with other devices.
   - Running on Microsoft Windows Vista Home Premium 64-Bit, it offers a user-friendly operating system.

2. Apple MacBook 2.4GHz Intel Core 2 Duo Silver Notebook Computer - MB467LLA:
   - Featuring a 2.4GHz Intel Core 2 Duo Processor, this MacBook provides fast and efficient performance.
   - The 13" LED-Backlit Glossy Widescreen Display offers a compac